## Day 5: Hydrothermal Venture

You come across a field of hydrothermal vents on the ocean floor! These vents constantly produce large, opaque clouds, so it would be best to avoid them if possible.

They tend to form in lines; the submarine helpfully produces a list of nearby lines of vents (your puzzle input) for you to review. For example:

```
0,9 -> 5,9
8,0 -> 0,8
9,4 -> 3,4
2,2 -> 2,1
7,0 -> 7,4
6,4 -> 2,0
0,9 -> 2,9
3,4 -> 1,4
0,0 -> 8,8
5,5 -> 8,2
```

Each line of vents is given as a line segment in the format x1,y1 -> x2,y2 where x1,y1 are the coordinates of one end the line segment and x2,y2 are the coordinates of the other end. These line segments include the points at both ends. In other words:

- An entry like 1,1 -> 1,3 covers points 1,1, 1,2, and 1,3.
- An entry like 9,7 -> 7,7 covers points 9,7, 8,7, and 7,7.

For now, only consider horizontal and vertical lines: lines where either x1 = x2 or y1 = y2.

So, the horizontal and vertical lines from the above list would produce the following diagram:

```
.......1..
..1....1..
..1....1..
.......1..
.112111211
..........
..........
..........
..........
222111....
```

In this diagram, the top left corner is 0,0 and the bottom right corner is 9,9. Each position is shown as the number of lines which cover that point or . if no line covers that point. The top-left pair of 1s, for example, comes from 2,2 -> 2,1; the very bottom row is formed by the overlapping lines 0,9 -> 5,9 and 0,9 -> 2,9.

To avoid the most dangerous areas, you need to determine the number of points where at least two lines overlap. In the above example, this is anywhere in the diagram with a 2 or larger - a total of 5 points.

Consider only horizontal and vertical lines. At how many points do at least two lines overlap?

In [130]:
import copy
import numpy as np


def read_vent_input(filename):
    """
    Parse vent data input from .txt file

    """

    with open(filename, "r") as infile:
        vent_data_raw = [l.strip() for l in infile]

    vent_data = []

    for x in vent_data_raw:
        vent_data.append([list(map(int, y.split(","))) for y in x.split(" -> ")])

    return vent_data


def display_vent_diagram(vent_diagram):
    """
    Pretty printing of vent diagram

    """

    vent_diagram = [list(map(str, x)) for x in vent_diagram]

    [print("".join(x).replace("0", ".")) for x in vent_diagram]

    return 0


def create_vent_diagram(filename, display_diagram=False, diagonal=False):
    """
    Create vent diagram

    """

    vent_data = read_vent_input(filename)

    # Set up matrix to store diagram
    size_x = max([max(x[0][0], x[1][0]) for x in vent_data])
    size_y = max([max(x[0][1], x[1][1]) for x in vent_data])

    temp_x = [0] * (size_x + 1)
    vent_diagram = [copy.deepcopy(temp_x) for i in range(0, size_y + 1)]

    # For each datapoint in vent data, update vent diagram
    for v in vent_data:

        if (
            (v[0][0] == v[1][0])
            or (v[0][1] == v[1][1])
            or (diagonal and (abs(v[0][0] - v[1][0]) == abs(v[0][1] - v[1][1])))
        ):

            # There's probably a cleaner way of doing this 😤
            x1 = min(v[0][0], v[1][0])
            x2 = max(v[0][0], v[1][0])
            y1 = min(v[0][1], v[1][1])
            y2 = max(v[0][1], v[1][1])

            # Generate points in betweena

            if x1 == x2:
                vent_points = [[x1, y] for y in range(y1, y2 + 1)]
            elif y1 == y2:
                vent_points = [[x, y1] for x in range(x1, x2 + 1)]
            else:
                # Also consider diagonal vents 🤔
                x_sign = int(np.sign(v[1][0] - v[0][0]))
                x_points = [x for x in range(v[0][0], v[1][0] + x_sign, x_sign)]

                y_sign = int(np.sign(v[1][1] - v[0][1]))
                y_points = [y for y in range(v[0][1], v[1][1] + y_sign, y_sign)]

                vent_points = [[x, y] for x, y in zip(x_points, y_points)]

            for p in vent_points:
                vent_diagram[p[1]][p[0]] += 1

    if display_diagram:
        display_vent_diagram(vent_diagram)

    return vent_diagram


def count_overlaps(filename, diagonal=False):
    """
    Count overlapping points in vent diagram

    >>> count_overlaps("day5-example.txt")
    5

    >>> count_overlaps("day5-example.txt", True)
    12

    """

    vent_diagram = create_vent_diagram(filename, False, diagonal)

    num_overlaps = sum([sum([x > 1 for x in y]) for y in vent_diagram])

    return num_overlaps


import doctest

doctest.testmod()


TestResults(failed=0, attempted=2)

In [132]:
count_overlaps("day5-input.txt")

6572

## Part Two

Unfortunately, considering only horizontal and vertical lines doesn't give you the full picture; you need to also consider diagonal lines.

Because of the limits of the hydrothermal vent mapping system, the lines in your list will only ever be horizontal, vertical, or a diagonal line at exactly 45 degrees. In other words:

- An entry like 1,1 -> 3,3 covers points 1,1, 2,2, and 3,3.
- An entry like 9,7 -> 7,9 covers points 9,7, 8,8, and 7,9.

Considering all lines from the above example would now produce the following diagram:

```
1.1....11.
.111...2..
..2.1.111.
...1.2.2..
.112313211
...1.2....
..1...1...
.1.....1..
1.......1.
222111....
```

You still need to determine the number of points where at least two lines overlap. In the above example, this is still anywhere in the diagram with a 2 or larger - now a total of 12 points.

Consider all of the lines. At how many points do at least two lines overlap?

In [133]:
count_overlaps("day5-input.txt", True)

21466